In [93]:
#Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


#Time
from time import sleep

import pandas as pd
import itertools

In [62]:
def setupLinkedIn(email, password,sleep_time):
    #Setup   
    selEmail = driver.find_element(By.ID, 'session_key')
    selPW = driver.find_element(By.ID, 'session_password')
    selEmail.send_keys(email)
    selPW.send_keys(password)
    selPW.send_keys(Keys.RETURN)

    sleep(sleep_time)


In [69]:
def goToPage(page_number, sleep_time):
    #Go to the initial page
    baseUrl = 'https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22101174742%22%5D&origin=GLOBAL_SEARCH_HEADER'
    pageUrl = baseUrl + f'&page={page_number}'
    driver.get(pageUrl)
    
    sleep(sleep_time)
    

In [64]:
def getCompanyPageLinks(sleep_time):
    """
    Requirements: be on a company search page
    Should be filtered by country Canada first, but will work on any company search page
    """
    companyWebElements = driver.find_elements(By.XPATH, "//span[@class='entity-result__title-text t-16']//a[@href]")
    companyLinks = []
    for elems in companyWebElements:
        companyLinks.append(elems.get_attribute('href'))
        
    sleep(sleep_time)
    return companyLinks

In [71]:
def loopThroughPageLinks(page_number_list: list, sleep_time):
    
    ###
    # First loop: get all of the company links
    ###
    all_links = []
    #Loop through every page
    for page_number in page_number_list:
        
        goToPage(page_number = page_number, sleep_time = sleep_time)
        
        all_links.append(getCompanyPageLinks(sleep_time))
    
    return all_links

In [117]:
def getAboutText(company_link, sleep_time):
    """
    Gets the about text and company name
    """
    #Go to company about page
    about_link = company_link + 'about/'
    driver.get(about_link)

    sleep(sleep_time)
    aboutText = driver.find_element(By.XPATH, "//div[@class='ember-view']//section[@class='artdeco-card p5 mb4']")
    companyName = driver.find_element(By.XPATH, "//h1[@id='ember31']")
    return aboutText.text, companyName.text

In [178]:
def loopThroughAboutTexts(company_links_nested, sleep_time):
    output_list = []
    for i, link in enumerate(list(itertools.chain.from_iterable(company_links_nested))):
        abouttext, companyname = getAboutText(company_link = link, sleep_time = sleep_time)
        output_list.append({'id' : i,
                         'Name' : companyname,
                         'Link' : link,
                         'aboutText' : abouttext})
    return output_list

In [179]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://linkedin.com')
sleep(3)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\louis\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\louis\AppData\Local\Temp\ipykernel_1960\1904361333.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [180]:
#Sign in to Feed Page
setupLinkedIn(email = 'louis.dhulst@mail.mcgill.ca', password = 'zvkDry%ZdCX5Q7e',sleep_time = 5)

links = loopThroughPageLinks(page_number_list = list(range(1,2)), sleep_time = 5)

linkedIn_text = loopThroughAboutTexts(company_links_nested = links, sleep_time = 5)

In [196]:
desired_fields = ['Overview', 'Website','Industry','Company size','Headquarters','Specialties']
cleanedText = []
pointer = None


for company_dict in linkedIn_text:
    #Initialize empty dict with fields
    cleanedText.append({'Name':company_dict['Name'],
                       'LinkedIn Link':company_dict['Link']})
    cleanedText[-1].update({key: None for key in desired_fields})
    
    for text in company_dict['aboutText'].split("\n"):

        #Continue for empty string
        if not text:
            continue

        #Update pointer
        if text in desired_fields:
            pointer = text
            continue
        #Put in text  
        if pointer:
            if not cleanedText[-1][pointer]:
                cleanedText[-1][pointer] = text
            else:
                cleanedText[-1][pointer] += ' ' + text
        

In [198]:
scrapedAbouts = pd.DataFrame(cleanedText)

In [199]:
#Save to excel
scrapedAbouts.to_excel("linkedIn-scrape.xlsx")